# Sprint アンサンブル学習

### 小さなデータセットの用意

以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import copy
from collections import defaultdict

In [2]:
import matplotlib

In [3]:
df = pd.read_csv("../train.csv")
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = df.loc[:, ["SalePrice"]]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=2)


In [4]:
#対数変換
y_train=np.log(y_train)
y_train

array([[12.07539432],
       [11.79433792],
       [11.89818787],
       ...,
       [11.9511804 ],
       [13.00865926],
       [11.73606902]])

In [5]:
y_val = np.log(y_val)
y_val

array([[12.57418197],
       [12.08953883],
       [12.79135618],
       [12.19095901],
       [12.31940133],
       [12.30591798],
       [12.3327053 ],
       [12.21106019],
       [11.34450681],
       [12.66032792],
       [12.27839331],
       [12.24047407],
       [11.6784399 ],
       [11.89818787],
       [11.56171563],
       [12.32385568],
       [12.56723749],
       [12.55672952],
       [12.3883942 ],
       [12.56024446],
       [12.64432758],
       [12.05815252],
       [12.06681058],
       [12.31043266],
       [12.99175343],
       [12.46997419],
       [11.60823564],
       [12.47716791],
       [12.22587527],
       [12.98804081],
       [12.66191396],
       [11.9511804 ],
       [12.5776362 ],
       [11.72803684],
       [12.08107616],
       [12.26434155],
       [11.7905572 ],
       [12.75995776],
       [12.76568843],
       [12.95512746],
       [12.06104687],
       [12.28995413],
       [11.9316358 ],
       [11.39639165],
       [11.58896015],
       [12

# 【問題1】ブレンディングのスクラッチ実装

ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

ブレンディングとは?

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [3]:
housing = pd.read_csv("../train.csv")

In [4]:
X = housing[["GrLivArea", "YearBuilt"]]
X.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [5]:
y = housing["SalePrice"]
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
(X_train, X_test,
 y_train, y_test) = train_test_split(
    X, y, test_size=0.3, random_state=0, shuffle=True
)

### part1

In [7]:
LM= linear_model.LinearRegression(normalize=True)
print(LM)
LM.fit(X_train, y_train)
LM_pred = LM.predict(X_test)
print("MSE(linear_model)",mean_squared_error(y_test, LM_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='rbf')
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = LM_pred*0.5+SV_pred*0.1+DR_pred*0.4
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)
MSE(linear_model) 2690647926.377603
MSE(SVR) 7119069689.683064
MSE(DecisionTreeRegressor) 2553443455.30812
MSE(mean) 2470929971.5410113


### part2 

In [8]:
from sklearn.linear_model import Lasso

Las= Lasso(alpha=0.1)
# print(Las)
Las.fit(X_train, y_train)
Las_pred = Las.predict(X_test)
print("MSE(Lasso)",mean_squared_error(y_test, Las_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='rbf',C=0.5)
# print(SV)
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=3)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = Las_pred*0.5+SV_pred*0.05+DR_pred*0.45
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

MSE(Lasso) 2690647942.4335794
MSE(SVR) 7128920068.0851145
MSE(DecisionTreeRegressor) 2553443455.30812
MSE(mean) 2413207951.9277987


### part3

In [9]:
from sklearn.linear_model import Ridge

Rid= Ridge(alpha=0.1)
# print(Las)
Rid.fit(X_train, y_train)
Rid_pred = Rid.predict(X_test)
print("MSE(Ridge)",mean_squared_error(y_test, Rid_pred))


sc = StandardScaler()
X_test_std = sc.fit_transform(X_test)
X_train_std = sc.fit_transform(X_train)
SV= SVR(kernel='linear', C=100)
# print(SV)
SV.fit(X_train_std, y_train)
SV_pred = SV.predict(X_test_std)
print("MSE(SVR)",mean_squared_error(y_test, SV_pred))


DR = DecisionTreeRegressor(max_depth=9)
DR.fit(X_train, y_train)
DR_pred = DR.predict(X_test)
print("MSE(DecisionTreeRegressor)",mean_squared_error(y_test, DR_pred))

mix_pred = np.vstack((LM_pred, SV_pred, DR_pred))
mix_pred.shape

mean_pred = Rid_pred*0.2+SV_pred*0.1+DR_pred*0.7
print("MSE(mean)",mean_squared_error(y_test, mean_pred))

MSE(Ridge) 2690647942.087792
MSE(SVR) 3389628361.3601527
MSE(DecisionTreeRegressor) 1922385503.137878
MSE(mean) 1838951096.4708827


# 【問題2】バギングのスクラッチ実装

バギングとは

バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

In [20]:
import xgboost as xgb

In [21]:
np.random.seed(1)
rand_index = np.random.choice(X_train.index,1022, replace=True)
np.unique(rand_index).shape

(649,)

In [22]:
np.random.seed(1)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True)
X_train1 = X_train.loc[rand_index]
y_train1 = y_train.loc[rand_index]

np.random.seed(2)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True)
X_train2 = X_train.loc[rand_index]
y_train2 = y_train.loc[rand_index]

np.random.seed(3)
rand_index = np.random.choice(X_train.index,X_train.shape[0], replace=True).tolist()
X_train3 = X_train.loc[rand_index]
y_train3 = y_train.loc[rand_index]


In [23]:
reg1 = GradientBoostingRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = GradientBoostingRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = GradientBoostingRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))
print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))

MSE(reg1) 1955856381.8803396
MSE(reg2) 2246120995.481195
MSE(reg2) 1725817235.2522635
MSE(mean) 1659935727.6287725


In [24]:
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = xgb.XGBRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = xgb.XGBRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))

print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))

MSE(reg1) 1955856381.8803396
MSE(reg2) 2468960358.254284
MSE(reg2) 1958559489.3734016
MSE(mean) 1741647524.5251908


In [25]:
reg1 = lgb.LGBMRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1 =reg1.predict(X_test)
print("MSE(reg1)",mean_squared_error(y_test, y_pred1))

reg2 = lgb.LGBMRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2 =reg2.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred2))

reg3 = lgb.LGBMRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3 =reg3.predict(X_test)
print("MSE(reg2)",mean_squared_error(y_test, y_pred3))
print("==================")
y_pred_mean = (y_pred1 + y_pred2 + y_pred3)/3
print("MSE(mean)",mean_squared_error(y_test, y_pred_mean))
y_pred3[1]

MSE(reg1) 1996989149.286145
MSE(reg2) 2217785612.142733
MSE(reg2) 2337926346.668943
MSE(mean) 2010251814.6958816


149261.18481853406

In [26]:
class Bagging:

    #     def __init__(self, n_clusters, max_iter, verbose=False, tol=5, seed=0):
    #         # ハイパーパラメータを属性として記録
    #         self.n_clusters = n_clusters
    #         self.seed = seed
    #         self.n_init = n_init
    #         self.max_iter = max_iter
    #         self.tol= tol
    #         self.verbose = verbose

    def fit(self, models, X, y):
        self.model_list = []
        for j, model in enumerate(models):
            np.random.seed(j)
            self.rand_index = np.random.choice(X.index, X.shape[0], replace=True)
            self.X_rand = X.loc[self.rand_index]
            self.y_rand = y.loc[self.rand_index]
            self.model_list.append(model.fit(self.X_rand, self.y_rand))

    def predict(self, X, y):
        print(X.shape[0])
        print(len(self.model_list))
        self.pred_data = np.zeros((X.shape[0], len(self.model_list)))
        for i, model in enumerate(self.model_list):
            self.pred = model.predict(X)
            self.pred_data[:, i] = self.pred
        self.final_pred = np.mean(self.pred_data, axis=1)
        print("MSE(mean)", mean_squared_error(y, self.final_pred))
        return self.final_pred

In [27]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), GradientBoostingRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

In [28]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1843004525.0391715


array([233606.53154969, 156557.12682114, 128544.76905402, 162812.42505345,
       119902.95170025,  93717.61151535, 202045.47302486, 123859.74040471,
       565346.0913109 , 148427.40559746])

In [29]:
models = [lgb.LGBMRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

In [30]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1905905323.963103


array([186616.01381421, 154654.02881744, 128076.41901312, 184057.70727728,
       128618.50371924,  91991.24366953, 202164.25872405, 126932.11921891,
       441678.27135167, 146946.96392326])

In [31]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), xgb.XGBRegressor(n_estimators=100,max_depth=5), xgb.XGBRegressor(n_estimators=100,max_depth=5)]
bg = Bagging()
bg.fit(models=models, X = X_train, y = y_train)

In [32]:
bg.predict(X_test, y_test)[:10]

438
3
MSE(mean) 1881306593.3967283


array([254122.953125  , 149782.68229167, 126505.11197917, 151397.40625   ,
       119442.3671875 ,  93480.6328125 , 199798.34375   , 121269.84635417,
       585070.83333333, 147071.07291667])

# 【問題3】スタッキングのスクラッチ実装

スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。

《学習時》


（ステージ 
0
 ）


学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。


（ステージ 
n
 ）


ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ 
0
 ）


テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
n
 ）


ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [54]:
from sklearn.model_selection import KFold

In [38]:
X1_train, X2_train, X3_train  = np.split(X_train, [340, 681])
y1_train, y2_train, y3_train  = np.split(y_train, [340, 681])

In [39]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_Gr = GradientBoostingRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_Gr =reg1_Gr.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_Gr))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_Gr = GradientBoostingRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_Gr =reg2_Gr.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_Gr))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_Gr = GradientBoostingRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_Gr =reg3_Gr.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_Gr))

y_train_Gr = np.hstack((y_pred1_reg1_Gr, y_pred2_reg2_Gr, y_pred2_reg2_Gr))

MSE(reg1) 2047990274.8480294
MSE(reg2) 2295774974.924504
MSE(reg2) 1458207763.1083522


In [40]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_LG = lgb.LGBMRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_LG =reg1_LG.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_LG))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_LG = lgb.LGBMRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_LG =reg2_LG.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_LG))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_LG = lgb.LGBMRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_LG =reg3_LG.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_LG))

y_train_LG = np.hstack((y_pred1_reg1_LG, y_pred2_reg2_LG, y_pred2_reg2_LG))

MSE(reg1) 1789291655.0779407
MSE(reg2) 2088962682.1700788
MSE(reg2) 1383718320.4924493


In [41]:
X_train1 = np.vstack((X2_train, X3_train))
y_train1 = np.hstack((y2_train, y3_train))

reg1_RF = RandomForestRegressor(n_estimators=100,max_depth=5).fit(X_train1, y_train1)
y_pred1_reg1_RF =reg1_RF.predict(X1_train)
print("MSE(reg1)",mean_squared_error(y1_train, y_pred1_reg1_RF))


X_train2 = np.vstack((X1_train, X3_train))
y_train2 = np.hstack((y1_train, y3_train))

reg2_RF = RandomForestRegressor(n_estimators=500,max_depth=3).fit(X_train2, y_train2)
y_pred2_reg2_RF =reg2_RF.predict(X2_train)
print("MSE(reg2)",mean_squared_error(y2_train, y_pred2_reg2_RF))


X_train3 = np.vstack((X1_train, X2_train))
y_train3 = np.hstack((y1_train, y2_train))


reg3_RF = RandomForestRegressor(n_estimators=250,max_depth=5).fit(X_train3, y_train3)
y_pred3_reg3_RF =reg3_RF.predict(X3_train)
print("MSE(reg2)",mean_squared_error(y3_train, y_pred3_reg3_RF))

y_train_RF = np.hstack((y_pred1_reg1_RF, y_pred2_reg2_RF, y_pred2_reg2_RF))

MSE(reg1) 1973940158.6693058
MSE(reg2) 2423912060.674026
MSE(reg2) 1182302514.416233


In [42]:
train_mix = np.vstack((y_train_Gr, y_train_LG, y_train_RF)).T
train_mix.shape

(1022, 3)

In [43]:
xgb.XGBRegressor
reg_end = xgb.XGBRegressor(n_estimators=250,max_depth=5).fit(train_mix, y_train)

In [44]:
X_test1_Gr =reg1_Gr.predict(X_test)
X_test2_Gr =reg2_Gr.predict(X_test)
X_test3_Gr = reg3_Gr.predict(X_test)

y_test_Gr = (X_test1_Gr + X_test2_Gr + X_test3_Gr) /3

In [46]:
y_test_Gr.shape

(438,)

In [47]:
X_test1_LG =reg1_LG.predict(X_test)
X_test2_LG =reg2_LG.predict(X_test)
X_test3_LG = reg3_LG.predict(X_test)

y_test_LG = (X_test1_LG + X_test2_LG + X_test3_LG) /3

In [48]:
y_test_LG.shape

(438,)

In [49]:
X_test1_RF =reg1_RF.predict(X_test)
X_test2_RF =reg2_RF.predict(X_test)
X_test3_RF = reg3_RF.predict(X_test)

y_test_RF = (X_test1_RF + X_test2_RF + X_test3_RF) /3

In [50]:
y_test_RF.shape

(438,)

In [51]:
X_test_end = np.vstack((y_test_Gr,y_test_LG,y_test_RF)).T
X_test_end.shape

(438, 3)

In [52]:
y_pred_end = reg_end.predict(X_test_end)
y_pred_end.shape

(438,)

In [53]:
print("MSE(Stacking)",mean_squared_error(y_test, y_pred_end))

MSE(Stacking) 3572015419.653251


In [55]:
class Stacking:
    def __init__(self, models, end_model):
        self.models = models
        self.end_model = end_model

    def fit(self, X, y, K, seed):
        self.K = K
        KF = KFold(n_splits=K, random_state=seed, shuffle=True)
        self.pred_data = np.array([])

        X = np.array(X)
        y = np.array(y)
        self.preds = np.array([])
        self.model_list = []
        for i, model in enumerate(self.models):
            for train_index, test_index in KF.split(X):

                self.model_list.append(model.fit(X[train_index], y[train_index]))
                y_pred = model.predict(X[test_index])

                self.preds = np.append(self.preds, y_pred)
        self.preds = self.preds.reshape(len(self.models), X.shape[0]).T
        self.end_model.fit(self.preds, y)

    #         print(self.end_model)

    def predict(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.test_array = np.array([])
        self.final_test = np.zeros((X.shape[0], self.K))
        for model in self.model_list:
            y_pred_test = model.predict(X)
            self.test_array = np.append(self.test_array, y_pred_test)
        #             print(self.test_array.shape,"test_array")
        self.test_array = self.test_array.reshape(
            len(self.models) * self.K, X.shape[0]
        ).T
        print(self.test_array.shape, "test_array")
        for j, i in enumerate(range(0, len(self.models) * self.K - self.K, self.K)):
            self.mean_pred = np.mean(self.test_array[:, i : i + self.K], axis=1)
            self.final_test[:, j] = self.mean_pred
        self.final_pred = self.end_model.predict(self.final_test)
        print(self.final_test.shape, "final_test")
        return self.final_pred

In [56]:
models = [xgb.XGBRegressor(n_estimators=100,max_depth=5), GradientBoostingRegressor(n_estimators=100,max_depth=5), lgb.LGBMRegressor(n_estimators=100,max_depth=5)]

In [57]:
St = Stacking(models,end_model=GradientBoostingRegressor(n_estimators=100,max_depth=5))
St.fit(X=X_train, y=y_train,K=3,seed=0)
St.pred_data.shape

(0,)

In [58]:
y_pred_end = St.predict(X_test, y_test)
print("MSE(Stacking)",mean_squared_error(y_test, y_pred_end))

(438, 9) test_array
(438, 3) final_test
MSE(Stacking) 13057076528.599543
